# bad_words 기반 마스킹 & 한글 프롬프트 적용 버전


In [ ]:
!pip install -q transformers evaluate sentencepiece
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q --upgrade transformers
import torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 130.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files

print("먼저 순화 데이터(.jsonl)를 업로드하세요.")
uploaded1 = files.upload()
file_path = next(fname for fname in uploaded1 if fname.endswith(".jsonl"))
print("Raw data file:", file_path)

먼저 순화 데이터(.jsonl)를 업로드하세요.


Saving 순화표현모델 데이터_리라이팅완료.jsonl to 순화표현모델 데이터_리라이팅완료.jsonl
Raw data file: 순화표현모델 데이터_리라이팅완료.jsonl


In [ ]:
print("이제 bad_words(.txt)를 업로드하세요.")
uploaded2 = files.upload()
bad_words_path = next(fname for fname in uploaded2 if fname.endswith(".txt"))
print("Bad words file:", bad_words_path)

이제 bad_words(.txt)를 업로드하세요.


Saving bad_words.txt to bad_words.txt
Bad words file: bad_words.txt


In [ ]:
# ─── Colab 셀 4: 라이브러리 임포트 & 모델 로드 ───
import json, re
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments, Seq2SeqTrainer
)
import evaluate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

MODEL_NAME = "wisenut-nlp-team/KoT5-base"
tokenizer  = AutoTokenizer.from_pretrained(MODEL_NAME)
model      = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)

Using device: cuda


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
# bad_words.txt 로드
with open(bad_words_path, "r", encoding="utf-8") as f:
    bad_patterns = [re.escape(w.strip()) for w in f if w.strip()]
print("Bad patterns:", len(bad_patterns))

Bad patterns: 654


In [ ]:
# ─── Colab 셀 5: 데이터 로드 & 2-step 전처리 ───
with open(file_path, "r", encoding="utf-8") as f:
    raw_data = [json.loads(line) for line in f if line.strip()]
print("Total samples:", len(raw_data))

def mask_toxic(example):
    last = example["context"][-1]
    for pat in bad_patterns:
        last = re.sub(pat, "[TOXIC]", last)
    return example["context"][:-1] + [last]

def preprocess_2step(example):
    masked = mask_toxic(example)
    prefix = "마스킹된 부분을 부드럽게 바꿔줘:"
    txt = " ".join([prefix] + masked)
    return {"input_text": txt, "target_text": example["output"]}

processed = [preprocess_2step(d) for d in raw_data]
print("Example:", processed[0])

Total samples: 3100
Example: {'input_text': '마스킹된 부분을 부드럽게 바꿔줘: A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴 B: 나도 거의 뛰어서 옴;; A: 수학쌤 오늘 또 지옥 시간일 듯 B: 그 [TOXIC] 늙은이 목소리만 들어도 현타 옴', 'target_text': 'B: 수학쌤 목소리 너무 지루하긴 해'}


In [ ]:
# ─── Colab 셀 6: train/test split & Dataset 정의 ───
from sklearn.model_selection import train_test_split

train_list, test_list = train_test_split(processed, test_size=0.1, random_state=42)

class Rewrite2stepDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data      = data
        self.tokenizer = tokenizer
        self.max_len   = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        item = self.data[i]
        enc = self.tokenizer(
            item["input_text"],
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        lbl = self.tokenizer(
            text_target=item["target_text"],
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"].squeeze()
        lbl[lbl == self.tokenizer.pad_token_id] = -100
        batch = {k: v.squeeze() for k, v in enc.items()}
        batch["labels"] = lbl
        return batch

train_ds = Rewrite2stepDataset(train_list, tokenizer)
test_ds  = Rewrite2stepDataset(test_list,  tokenizer)
collator = DataCollatorForSeq2Seq(tokenizer, model)

print("Train/Test sizes:", len(train_ds), len(test_ds))

Train/Test sizes: 2790 310


In [ ]:
# ─── Colab 셀 7: TrainingArguments & Trainer (compute_metrics 생략) ───
args = Seq2SeqTrainingArguments(
    output_dir="/content/kot5-2step",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=64,
    generation_num_beams=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps=100,
    save_total_limit=2,
    lr_scheduler_type="linear",
    logging_dir="/content/logs_2step",
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    tokenizer=tokenizer,
    data_collator=collator
)

print("Trainer ready.")

Trainer ready.


/tmp/ipython-input-820605648.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()
trainer.save_model("/content/kot5-2step")
tokenizer.save_pretrained("/content/kot5-2step")

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss
500,5.265100
1000,4.467200
1500,3.971300
2000,3.732500


('/content/kot5-2step/tokenizer_config.json',
 '/content/kot5-2step/special_tokens_map.json',
 '/content/kot5-2step/tokenizer.json')

In [ ]:
# ─── Colab 셀 9: 수동 평가 ───
import evaluate
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

preds, refs = [], []
for ex in test_list:
    inp = ex["input_text"]
    out = model.generate(
        **tokenizer(inp, return_tensors="pt", truncation=True, max_length=256, padding="max_length").to(device),
        max_length=64,
        num_beams=4,
        early_stopping=True
    )[0]
    preds.append(tokenizer.decode(out, skip_special_tokens=True))
    refs.append(ex["target_text"])

bleu_score  = bleu.compute(predictions=preds, references=[[r] for r in refs])["bleu"]
rouge_score = rouge.compute(predictions=preds, references=refs, use_stemmer=True)["rougeL"]

print(f"\n▶ Final BLEU:  {bleu_score:.4f}")
print(f"▶ Final ROUGE-L: {rouge_score:.4f}")


▶ Final BLEU:  0.0000
▶ Final ROUGE-L: 0.0140
